In [50]:
import torch
from botorch.models import MultiTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition.monte_carlo import qUpperConfidenceBound
from botorch.acquisition.objective import GenericMCObjective
import numpy as np
import time
import datetime
from botorch.optim import optimize_acqf
from torch.utils.tensorboard import SummaryWriter
import os

In [2]:
tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

### 注意
- metricsの計算およびobjectiveも最小化基準で設計すること

In [66]:
import abc

class IMetric(abc.ABC):
    def __init__(self):
        self.optimal_metric = None

    def parameter_parser(self, x):
        # x: Tensor[D]
        # ここでは簡単のためにパラメーターベクトルからパラメーターを取り出す
        gamma = x[0]
        beta = x[1]
        shadow = x[2]
        return gamma, beta, shadow
    def get_image_from_parameter(self, x):
        # x: Tensor[D]
        # ここは実際にはopenCV or simulatorによってパラメーターから画像を生成する
        gamma, beta, shadow = self.parameter_parser(x)
        
        image = gamma + beta + shadow

        return image
    @abc.abstractmethod
    def calc(self, x):
        # 必ずself.__update_optimal_metricを呼び出すこと
        raise NotImplementedError()
    
    @abc.abstractmethod
    def update_optimal_metric(self, y):
        # yを条件判定した上で最適値を更新する
        raise NotImplementedError()

class Metric0(IMetric):
    
    def calc(self, x):
        image = self.get_image_from_parameter(x)
        y = 3 * image**3 + 2 * image**2 + image + 1
        self.update_optimal_metric(y)
        return y

    def update_optimal_metric(self, y):
        # yを条件判定した上で最適値を更新する
        if self.optimal_metric is None:
            self.optimal_metric = y
        elif self.optimal_metric > y:
            self.optimal_metric = y

class Metric1(IMetric):
    
    def calc(self, x):
        image = self.get_image_from_parameter(x)
        y = - 2 * image**2 + image**2 + image + 1
        self.update_optimal_metric(y)
        return y
    
    def update_optimal_metric(self, y):
        # yを条件判定した上で最適値を更新する
        if self.optimal_metric is None:
            self.optimal_metric = y
        elif self.optimal_metric > y:
            self.optimal_metric = y

    
    

In [67]:
class Metrics():
    def __init__(self):
        self.metrics = {
            "metric0": Metric0(),
            "metric1": Metric1(),
        }
        self.metric_idxs = {
            0: self.metrics["metric0"],
            1: self.metrics["metric1"],
        }
        self.n_metrics = len(self.metrics)
        self.objective = GenericMCObjective(self.aggregator)

    def aggregator(self, samples, X=None):
        # multi-taskなので回帰モデルは複数のyを出力する
        # 獲得関数ではそれを集約し,スカラにする必要がある
        # 今回は全メトリクスの和を考える
        # samples: Tensor[sample_shape?, batch, 取り出す次のサンプル数, タスク数]
        
        return samples.sum(dim=-1)
    
    def get_optimal_metrics(self):
        optimal_metrics = { metric_name: metric.optimal_metric for metric_name, metric in self.metric_idxs.items()}
    
    def __call__(self, X):
        # X: Tensor[N, D+1]
        # return: Tensor[N, 1]
        return torch.stack([self.metric_idxs[x[-1].item()].calc(x[:-1]) for x in X]).unsqueeze(dim=1)
    
metrics = Metrics()

In [68]:
class Parameter():
    def __init__(self, bound, tkwargs):
        # bounds: 2d-list
        self.tkwargs = tkwargs
        self.bound = bound

    def random_sample(self, num):
        return (self.bound[1][0] - self.bound[0][0]) * torch.rand(num, 1, **self.tkwargs) + self.bound[0][0]
    
    def get_bound(self):
        return self.bound



class Parameters:
    def __init__(self, tkwargs):
        self.tkwargs = tkwargs
        self.param_idx = {
            "param0": 0,
            "param1": 1,
            "param2": 2,
        }
        self.params = {
            "param0": Parameter([[-20], [20]], tkwargs),
            "param1": Parameter([[-5], [5]], tkwargs),
            "param2": Parameter([[-5], [40]], tkwargs),
        }
        self.n_params = len(self.params)
        self.bounds = torch.tensor(np.concatenate([param.bound for param in self.params.values()], axis=1), **self.tkwargs)
        self.n_params = len(self.params)

    def random_sample(self, q):
        return torch.stack([param.random_sample(q) for param in self.params.values()], dim=1).squeeze(dim=2)
        
parameters = Parameters(tkwargs)

In [69]:
# 学習ハイパラ
n_iter = 100
n_sample_per_iter = 2

In [71]:
# Tensorboard
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
log_dir = f"./runs/{now.strftime('%Y%m%d%H%M%S')}"
gp_writer = SummaryWriter(log_dir=os.path.join(log_dir, "gp"))
rand_writer = SummaryWriter(log_dir=os.path.join(log_dir, "rand"))

# 範囲の保持やランダムサンプリングを担当するオブジェクト
parameters = Parameters(tkwargs)
# メトリクス値の計算や最大値の保持を担当するオブジェクト
gp_metrics = Metrics()
rand_metrics = Metrics()

# 初期サンプルの作成
init_params = parameters.random_sample(2)
# メトリクス（タスクの数）だけ縦に積み、タスク番号を最終列に追加
n_metrics = metrics.n_metrics
train_params = torch.cat([torch.cat((init_params, torch.tensor([[i] for _ in range(init_params.shape[0])], **tkwargs)), dim=1) for i in range(n_metrics)])
print(f"initial params: \n{train_params}")

# 初期学習データ
gp_train_params, gp_train_metrics = train_params, gp_metrics(train_params)
rand_train_params, rand_train_metrics = train_params, rand_metrics(train_params)

# tensorboardへ記録
for metric_name in gp_metrics.metrics.keys():
    gp_writer.add_scalar(f"{metric_name}", gp_metrics.metrics[metric_name].optimal_metric, 0)
    rand_writer.add_scalar(f"{metric_name}", rand_metrics.metrics[metric_name].optimal_metric, 0)

for iter in range(n_iter):

    torch.cuda.empty_cache()
    gp_t0 = time.time()
    # モデルの更新
    gp = MultiTaskGP(gp_train_params, gp_train_metrics, task_feature=-1)
    gp_mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(gp_mll, options={"maxiter": 3000, "lr": 0.01, "disp": False})
    # 獲得関数の計算
    qUCB = qUpperConfidenceBound(gp, beta=0.1, objective=metrics.objective)
    # 獲得関数の最適化
    gp_candidates, _ = optimize_acqf(
        acq_function=qUCB,
        bounds=parameters.bounds,
        q=n_sample_per_iter,
        num_restarts=10,
        raw_samples=128,  # used for intialization heuristic
        options={"batch_limit": 1, "maxiter": 200, "init_batch_limit": 1},
    )
    gp_t1 = time.time()
    # 得られた探索点にタスク番号を追加
    gp_candidates = torch.cat([torch.cat((gp_candidates, torch.tensor([[i] for _ in range(gp_candidates.shape[0])], **tkwargs)), dim=1) for i in range(n_metrics)])
    gp_candidates_metrics = gp_metrics(gp_candidates)
    # 学習データの更新
    gp_train_params = torch.cat((gp_train_params, gp_candidates), dim=0)
    gp_train_metrics = torch.cat((gp_train_metrics, gp_candidates_metrics), dim=0)

    rand_t0 = time.time()
    rand_candidates = parameters.random_sample(q=n_sample_per_iter)
    # 得られた探索点にタスク番号を追加
    rand_candidates = torch.cat([torch.cat((rand_candidates, torch.tensor([[i] for _ in range(rand_candidates.shape[0])], **tkwargs)), dim=1) for i in range(n_metrics)])
    rand_candidates_metrics = rand_metrics(rand_candidates)
    # 学習データの更新(学習しないけど...)
    rand_train_params = torch.cat((rand_train_params, rand_candidates), dim=0)
    rand_train_metrics = torch.cat((rand_train_metrics, rand_candidates_metrics), dim=0)
    rand_t1 = time.time()

    # tensorboardへ記録
    for metric_name in gp_metrics.metrics.keys():
        gp_writer.add_scalar(f"{metric_name}", gp_metrics.metrics[metric_name].optimal_metric, iter+1)
        rand_writer.add_scalar(f"{metric_name}", rand_metrics.metrics[metric_name].optimal_metric, iter+1)

        

    print(f"gp_time: {gp_t1 - gp_t0}")
    print(f"rand_time: {rand_t1 - rand_t0}")

initial params: 
tensor([[ 2.8715,  0.1486, 15.9543,  0.0000],
        [ 3.0004,  4.7735, 31.0274,  0.0000],
        [ 2.8715,  0.1486, 15.9543,  1.0000],
        [ 3.0004,  4.7735, 31.0274,  1.0000]], device='cuda:0',
       dtype=torch.float64)


/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:148: OptimizeWarning: Unknown solver options: lr
  warnings.warn(w.message, w.category)
/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:148: OptimizeWarning: Unknown solver options: lr
  warnings.warn(w.message, w.category)


gp_time: 6.022202253341675
rand_time: 0.0008833408355712891


/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:148: OptimizeWarning: Unknown solver options: lr
  warnings.warn(w.message, w.category)


gp_time: 3.470698118209839
rand_time: 0.0010285377502441406


...

gp_time: 3.8607258796691895
rand_time: 0.0009644031524658203


/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:148: OptimizeWarning: Unknown solver options: lr
  warnings.warn(w.message, w.category)
/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/optim/optimize.py:287: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), NumericalWarning('A not p.d., added jitter of 1.0e-07 to the diagonal'), NumericalWarning('A not p.d., added jitter of 1.0e-06 to the diagonal'), NumericalWarning('Runtime Error when computing Cholesky decomposition: Matrix not positive definite after repeatedly adding jitter up to 1.0e-06.. Using symeig method.'), NumericalWarning('A not p.d., added jitter of 1.0e-08 to the diagonal'), NumericalWarning('A not p.d., added jitter of 1.0e-07 to the diagonal'), NumericalWarning('A not p.d., added jitter of 1.0e-0

gp_time: 4.308732271194458
rand_time: 0.0008935928344726562


/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:148: OptimizeWarning: Unknown solver options: lr
  warnings.warn(w.message, w.category)
/home/shibutani/.local/share/virtualenvs/BoTorch_Trial-UdUIGeOu/lib/python3.7/site-packages/botorch/fit.py:155: RuntimeWarning: Fitting failed on all retries.
  warnings.warn("Fitting failed on all retries.", RuntimeWarning)


NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-06.

## 気づき
- 一応GPのほうがよりメトリクスを最小化できている
- マルチタスクのときは行列サイズが大きくなってしまうため、行列の正定値性に関して問題が起こりやすそう
- このことから、SingleTaskGPリストのほうが良さそう